<a href="https://colab.research.google.com/github/karma-os/Kaggle_study/blob/main/Spooky2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Approaching Any NLP Problem on kaggle

In [ ]:
import nltk
nltk.download("stopwords")
nltk.download("punkt")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from tqdm import tqdm
from sklearn.svm import SVC
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.layers import Embedding, BatchNormalization
from tensorflow.keras import utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from tensorflow.keras.layers import GlobalMaxPool1D, Conv1D, MaxPool1D, Flatten, Bidirectional, SpatialDropout1D
from tensorflow.keras.preprocessing import sequence, text
from tensorflow.keras.callbacks import EarlyStopping
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words("english")

In [ ]:
import os
need= "drive/MyDrive/Kaggle_kernel/data/spooky"

if os.getcwd() == "/content":
    chdir = os.path.join(os.getcwd(),need)
    os.chdir(need)

In [ ]:
#Linux switch command : 
"""
-f  freshen existing files, create none
-n  never overwrite existing files         
-q  quiet mode (-qq => quieter)
-o  overwrite files WITHOUT prompting 
"""

!unzip -o train.zip 
!unzip -o test.zip
!unzip -o sample_submission.zip

Archive:  train.zip
  inflating: train.csv               
Archive:  test.zip
  inflating: test.csv                
Archive:  sample_submission.zip
  inflating: sample_submission.csv   


In [ ]:
train = pd.read_csv("train.csv")
test  = pd.read_csv("test.csv")
sample = pd.read_csv("sample_submission.csv")

In [ ]:
# Text clf with 3 diff cases.
"""
Multi-class log loss
"""

def multiclass_logloss(actual, predicted, eps = 1e-15):
    """Multiclass version of log-loss matric.
    :param actual: Array containing the actual target classes
    :param predicted: Matrix with clas predictions, one prob per class
    """

    # Convert 'actual' to binary if it's not already:
    if len(actual.shape) == 1:
        actual2 = np.zeros((actual.shape[0], predicted.shape[1]))
        for i,val in enumerate(actual):
            actual2[i,val] = 1
        actual = actual2
    clip = np.clip(predicted, eps, 1-eps)
    rows = actual.shape[0]
    vsota = np.sum(actual * np.log(clip))
    return -1.0 / rows * vsota 



In [ ]:
#Label encoding to convert text labels to 0,1,2
lbl_enc = preprocessing.LabelEncoder()
y = lbl_enc.fit_transform(train.author.values)

In [ ]:
xtrain, xvalid, ytrain,yvalid = train_test_split(train.text.values, y,
                                                 stratify = y,
                                                 random_state = 42,
                                                 test_size = 0.1, 
                                                 shuffle = True)

In [ ]:
print(xtrain.shape)
print(xvalid.shape)

(17621,)
(1958,)


# Building Basic Models

**TF-IDF**

In [ ]:
#help(TfidfVectorizer)

"""
Params 복습

min_df

max_features

strip_accent : Remove accents and perform character normalization

analyzer : Feature made of word or n-grams?

token_pattern : '\w{1,}'

-> \w  : used to find a word character(a-z, A-Z, 0-9, character)
이런 w를 metacharacter라고 하나봐.

{n,m} : preceding character가 n~m회 반복.

그러면 결국 이 Token pattern은 word character가 1회 이상인 것.

디폴트 옵션은 \b\w\w+\b 라고 한다.


####
The default regexp selects tokens of 2
 or more alphanumeric characters (punctuation is completely ignored
 and always treated as a token separator).
####


ngram_range : ex) (1,3) : unigram, bigram, 3-gram

use_idf : Enable IDF reweighting.

smooth_idf : Prevents zero division

sublinear_tf : Replace tf with 1 + log(tf)

stop_words


"""



"\nParams 복습\n\nmin_df\n\nmax_features\n\nstrip_accent : Remove accents and perform character normalization\n\nanalyzer : Feature made of word or n-grams?\n\ntoken_pattern : '\\w{1,}'\n\n-> \\w  : used to find a word character(a-z, A-Z, 0-9, character)\n이런 w를 metacharacter라고 하나봐.\n\n-> {n,m} : preceding character가 n~m회 반복.\n\n그러면 결국 이 Token pattern은 word character가 1회 이상인거겠지.\n\n\n####\nThe default regexp selects tokens of 2\n or more alphanumeric characters (punctuation is completely ignored\n and always treated as a token separator).\n####\n\n\nngram_range : ex) (1,3) : unigram, bigram, 3-gram\n\nuse_idf : Enable IDF reweighting.\n\nsmooth_idf : Prevents zero division\n\nsublinear_tf : Replace tf with 1 + log(tf)\n\nstop_words\n\n\n"

In [ ]:
# Always start with these features. They work (almost) everytime!
tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = 'english')

# Fitting TF-IDF to both training and test sets (semi-supervised learning)
tfv.fit(list(xtrain) + list(xvalid))
xtrain_tfv =  tfv.transform(xtrain) 
xvalid_tfv = tfv.transform(xvalid)

In [ ]:
# Fitting a simple Logistic Regression on TFIDF
clf = LogisticRegression(C=1.0, max_iter = 500)
clf.fit(xtrain_tfv, ytrain)
predictions = clf.predict_proba(xvalid_tfv)

print("Case : LR on TFIDF")
print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

Case : LR on TFIDF
logloss: 0.572 


 **Let's improve it**

 











In [ ]:
ctv = CountVectorizer(analyzer = "word", token_pattern = r"\w{1,}",
                      ngram_range = (1,3), stop_words = "english")

#Fitting CountVectorizer to training, test sets

ctv.fit(list(xtrain)+list(xvalid))
xtrain_ctv = ctv.transform(xtrain)
xvalid_ctv = ctv.transform(xvalid)


In [ ]:
clf = LogisticRegression(C=1.0, max_iter = 500)

clf.fit(xtrain_ctv, ytrain)
predictions = clf.predict_proba(xvalid_ctv)

print("Case : LR on CountVectorizer")
print("logloss: %0.3f" %multiclass_logloss(yvalid,predictions))



Case : LR on CountVectorizer
logloss: 0.527


In [ ]:
#Fitting a simple NB on TFIDF

clf = MultinomialNB()
clf.fit(xtrain_tfv, ytrain)
predictions = clf.predict_proba(xvalid_tfv)

print("Case: NB on TFIDF")
print("logloss: %0.3f" %multiclass_logloss(yvalid,predictions))





Case: NB on TFIDF
logloss: 0.578


In [ ]:
#Fitting a simple NB on CountVetcorizer

clf = MultinomialNB()
clf.fit(xtrain_ctv, ytrain)
predictions = clf.predict_proba(xvalid_ctv)

print("Case : NB on CountVectorizer")
print("logloss: %0.3f" %multiclass_logloss(yvalid,predictions))

Case : NB on CountVectorizer
logloss: 0.485


**SVM**
Some people love SVM, so we'll do SVM.

Before SVm, we'll perform

SVD(for computation)
Standarization(compulsory)

In [ ]:
#Apply SVD. 120~200 components are good enough

svd = decomposition.TruncatedSVD(n_components = 120)
svd.fit(xtrain_tfv)
xtrain_svd = svd.transform(xtrain_tfv)
xvalid_svd = svd.transform(xvalid_tfv)

#Scale the data from SVD. Rename Variable to reuse without sacling

scl = preprocessing.StandardScaler()
scl.fit(xtrain_svd)
xtrain_svd_scl = scl.transform(xtrain_svd)
xvalid_svd_scl = scl.transform(xvalid_svd)

In [ ]:
#Apply SVM.
clf = SVC(C= 1.0,probability = True)
clf.fit(xtrain_svd_scl, ytrain)
predictions = clf.predict_proba(xvalid_svd_scl)

print("Case : SVM on TFIDF")
print("logloss: %0.3f" %multiclass_logloss(yvalid, predictions))




In [ ]:
# Fitting simple xgb on tf-idf

clf = xgb.XGBClassifier(max_depth = 7, n_estimators = 200, 
                        colsample_bytree = 0.8, subsample = 0.8, nthread = 10, learning_rate = 0.1)
clf.fit(xtrain_tfv.tocsc(),ytrain)
predictions = clf.predict_proba(xvalid_tfv.tocsc())


#tocsc(): compressed sparse column format

print("Case : XGB on TFIDF")
print("logloss: %0.3f" %multiclass_logloss(yvalid, predictions))


In [ ]:
# Fitting simple xgb on count

clf = xgb.XGBClassifier(max_depth = 7, n_estimators = 200, 
                        colsample_bytree = 0.8, subsample = 0.8, nthread = 10, learning_rate = 0.1)
clf.fit(xtrain_ctv.tocsc(),ytrain)
predictions = clf.predict_proba(xvalid_ctv.tocsc())


#tocsc(): compressed sparse column format

print("Case : XGB on TFIDF")
print("logloss: %0.3f" %multiclass_logloss(yvalid, predictions))

In [ ]:
# Fitting a simple xgboost on tf-idf svd features

clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(xtrain_svd, ytrain)
predictions = clf.predict_proba(xvalid_svd)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

In [ ]:
# Fitting a simple xgboost on tf-idf svd features
clf = xgb.XGBClassifier(nthread=10)
clf.fit(xtrain_svd, ytrain)
predictions = clf.predict_proba(xvalid_svd)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

**Grid Search**

Parameter looping

In [ ]:
mll_scorer = metrics.make_scorer(multiclass_logloss, greater_is_better= False, needs_proba = True)

In [ ]:
#pipeline : SVD -> Scaling -> LR

#Initialize svd

svd = TruncatedSVD()

#Initialize standard scaler

scl = preprocessing.StandardScaler()

#We'll use LR here.

lr_model = LogisticRegression()

#create Pipeline.

clf = pipeline.Pipeline([("svd",svd), ("scl",scl),("lr",lr_model)])


In [ ]:
#Param grid

param_grid = {"svd__n_components":[120,180], "lr__C":[0.1,1.0,10], "lr__penalty": ['l1','l2']}


In [ ]:
#Initialize Grid Search model

model = GridSearchCV(estimator = clf, param_grid = param_grid, scoring = mll_scorer,
                     verbose=10, n_jobs = -1, iid= True, refit = True, cv=2)


#Fit Grid Search model

model.fit(xtrain_tfv, ytrain)

In [ ]:
print("Best score: %0.3f" %model.best_score_)
print("Best params set:")
best_parameters = model.best_estimator_.get_params()

for param_name in sorted(param_grid.keys()):
    print("\t{0}: {1}".format(param_name, best_parameters[param_name]))

**LR tuning Score: similar to SVM ones**

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
nb_model = MultinomialNB()


clf = pipeline.Pipeline([('nb',nb_model)])

# parameter grid
param_grid = {'nb__alpha': [0.001, 0.01, 0.1, 1, 10, 100]}

# Initialize Grid Search Model
model = GridSearchCV(estimator=clf, param_grid=param_grid, scoring=mll_scorer,
                                 verbose=10, n_jobs=-1, iid=True, refit=True, cv=2)

# Fit Grid Search Model
model.fit(xtrain_tfv, ytrain)  # we can use the full data here but im only using xtrain. 
print("Best score: %0.3f" % model.best_score_)
print("Best parameters set:")
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))


**NB Grid search on TF : 0.57 -> 0.49**

#Word vectors

In [ ]:
!unzip -f glove.840B.300d.zip

Archive:  glove.840B.300d.zip


In [ ]:
ls

glove.840B.300d.txt  sample_submission.csv  test.csv  train.csv
glove.840B.300d.zip  sample_submission.zip  test.zip  train.zip


In [ ]:
#Load Glove vectors in a dictionary:

embeddings_index = {}
f = open("glove.840B.300d.txt")
for line in tqdm(f):
    try:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype="float32")
        embeddings_index[word] = coefs
    except:
        f.__next__()
f.close()

print(f"Found {len(embeddings_index)} word vectors")




2195997it [03:47, 9641.58it/s] 

Found 2195864 word vectors


In [ ]:
#CREATES NORMALIZED VECTOR FOR WHOLE SENTENCE

def sent2vec(s):
    words = str(s).lower()
    words = word_tokenize(words)
    words = [w for w in words if not w in stop_words]
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try: 
            M.append(embeddings_index[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis = 0)
    if type(v)!=np.array:
        return np.zeros(300)
    return v/np.sqrt((v**2).sum())



In [ ]:
#Create sentence ves using above ftn for train, val

xtrain_glove = [sent2vec(x) for x in tqdm(xtrain)]
xvalid_glove = [sent2vec(x) for x in tqdm(xvalid)]


100%|██████████| 1958/1958 [00:00<00:00, 2949.50it/s]


In [ ]:
xtrain_glove = np.array(xtrain_glove)
xvalid_glove = np.array(xvalid_glove)

In [ ]:
#Fitting simple xgboost on glove features

clf = xgb.XGBClassifier(nthread = 10, silent = False)
clf.fit(xtrain_glove, ytrain)
predictions = clf.predict_proba(xvalid_glove)


print("Case : Glove + XGBoost")

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))


Case : Glove + XGBoost
logloss: 1.088 


In [ ]:
# Fitting a simple xgboost on glove features
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1, silent=False)
clf.fit(xtrain_glove, ytrain)
predictions = clf.predict_proba(xvalid_glove)

print("Case: Glove + XGBoost other options ")
print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

Case: Glove + XGBoost other options 
logloss: 1.088 


## Deep Learning Era

LSTM, Simple Dense Network

---

In [ ]:
#Scale the data before any neural net:

scl = preprocessing.StandardScaler()
xtrain_glove_scl = scl.fit_transform(xtrain_glove)
xvalid_glove_scl  = scl.transform(xvalid_glove)




In [ ]:
from tensorflow.keras.utils import to_categorical

In [ ]:
#We need to binarize the label for NN

ytrain_enc = to_categorical(ytrain)
yvalid_enc = to_categorical(yvalid)


In [ ]:
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model

In [ ]:
#Create simple 3 layer Sequential neural net

input_tensor = Input(shape = (300,), name = "input")
x = Dense(300, activation = "relu", name = "dense1")(input_tensor)
x = Dropout(0.2)(x)
x = BatchNormalization()(x)

x = Dense(300,activation = "relu", name = "dense2")(x)
x = Dropout(0.3)(x)
x = BatchNormalization()(x)

x = Dense(3)(x)

output = Activation("softmax")(x)

model = Model(inputs= input_tensor, outputs = output)

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 300)]             0         
_________________________________________________________________
dense_9 (Dense)              (None, 300)               90300     
_________________________________________________________________
dropout_6 (Dropout)          (None, 300)               0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 300)               1200      
_________________________________________________________________
dense_10 (Dense)             (None, 300)               90300     
_________________________________________________________________
dropout_7 (Dropout)          (None, 300)               0         
_________________________________________________________________
batch_normalization_7 (Batch (None, 300)               1200  

In [ ]:
model.compile(loss="categorical_crossentropy",optimizer = "adam")

In [ ]:
#model fitting

model.fit(xtrain_glove_scl, y = ytrain_enc, batch_size = 64,
          epochs = 5, verbose = 1,
          validation_data=(xvalid_glove_scl, yvalid_enc))



Epoch 1/5
276/276 [==============================] - 3s 6ms/step - loss: 1.0891 - val_loss: 1.0881
Epoch 2/5
276/276 [==============================] - 2s 5ms/step - loss: 1.0882 - val_loss: 1.0887
Epoch 3/5
276/276 [==============================] - 1s 5ms/step - loss: 1.0881 - val_loss: 1.0878
Epoch 4/5
276/276 [==============================] - 2s 6ms/step - loss: 1.0880 - val_loss: 1.0883
Epoch 5/5
276/276 [==============================] - 2s 5ms/step - loss: 1.0879 - val_loss: 1.0875


## **Brief things about text.Tokenizer**

* 1. 문자 기반 인코딩

LISTEN -> L,I,S,T,E,N 아스키 이용하여 076, 073,...)

listen 과 silent은 비슷한 숫자로 인코딩. 

NN의 입력으론 부적합.

그래서 후술할 단어기반 tokenizer 이용.

* 2. 단어 기반 인코딩

I love my dog, I love my cat에서

tokenizer 쓰면

{"I":1, 'love':2, 'my' : 3 , 'dog' : 4, 'cat' : 5 }
가 되는데 이런걸 이용.

*num_words* = 몇개를 이용할 건지.

*fit_on_text* = 문자 데이터 입력 -> 리스트

*word_index* = tokenizer의 속성. {word : num} 형태의 dict반환

---

**texts_to_sequences** : 텍스트 안의 단어를 숫자 시퀀스로 변환.


**<font color = "red">oov_otken</font> = "OOV"**
- token화 안된것들을 "<OOV>"로 표시.

*pad_sequence* : 다른 개수의 단어인 문장들을 같은 길이로!

- padding: padding type지정.
- maxlen : sequence의 최대 길이 제한.
- truncating : 잘라낼 위치 지정.



In [ ]:
#Using LSTM

token = text.Tokenizer(num_words = None)
max_len = 70

token.fit_on_texts(list(xtrain)+list(xvalid))
xtrain_seq = token.texts_to_sequences(xtrain)
xvalid_seq = token.texts_to_sequences(xvalid)

#Zero pad the sequencees

xtrain_pad = sequence.pad_sequences(xtrain_seq, maxlen = max_len)
xvalid_pad = sequence.pad_sequences(xvalid_seq, maxlen = max_len)

word_index = token.word_index

In [ ]:
token

In [ ]:
#Create an embedding mtx for words in the dataset

embedding_matrix = np.zeros((len(word_index)+1, 300))

#embeddings_index는 glove vector 학습할 때 이미 가져왔었음!
for word, i in tqdm(word_index.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector





100%|██████████| 25943/25943 [00:00<00:00, 208090.13it/s]


# Some findings

- Embedding의 input_length에 맞게 Input 레이어를 설정해줘야함.

- RNN model은 SpatialDroupout1D랑 recurrent_dropout이라는걸 쓰나봄.


In [ ]:
from tensorflow.keras.layers import LSTM

In [ ]:
#A simple LSTM with glove


input_layer = Input(shape = (max_len,))
embedding = Embedding(len(word_index)+1, 300, weights = [embedding_matrix], 
              input_length = max_len, trainable = False)(input_layer)
x = SpatialDropout1D(0.3)(embedding)
x = LSTM(100, dropout = 0.3, recurrent_dropout = 0.3)(x)

x = Dense(1024, activation = "relu")(x)
x = Dropout(0.8)(x)

x = Dense(1024, activation = "relu")(x)
x = Dropout(0.8)(x)

x = Dense(3)(x)
output = Activation("softmax")(x)

model = Model(inputs = input_layer, outputs = output)

#Fit the model with early stopping callback

earlystop = EarlyStopping(monitor = "val_loss", min_delta = 0, patience = 3, verbose = 0, mode = "auto")


model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 70)]              0         
_________________________________________________________________
embedding_5 (Embedding)      (None, 70, 300)           7783200   
_________________________________________________________________
spatial_dropout1d_4 (Spatial (None, 70, 300)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               160400    
_________________________________________________________________
dense_18 (Dense)             (None, 1024)              103424    
_________________________________________________________________
dropout_12 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_19 (Dense)             (None, 1024)              1049

In [ ]:
model.compile(loss = "categorical_crossentropy", optimizer = "adam")

In [ ]:
model.fit(xtrain_pad, y=ytrain_enc, batch_size=512, epochs=100, 
          verbose=1, validation_data=(xvalid_pad, yvalid_enc),
          
          callbacks = [earlystop]
          
          )

Epoch 1/100
35/35 [==============================] - 56s 1s/step - loss: 1.0618 - val_loss: 0.9179
Epoch 2/100
35/35 [==============================] - 52s 1s/step - loss: 0.9016 - val_loss: 0.7536
Epoch 3/100
35/35 [==============================] - 52s 1s/step - loss: 0.8102 - val_loss: 0.7282
Epoch 4/100
35/35 [==============================] - 52s 1s/step - loss: 0.7834 - val_loss: 0.6963
Epoch 5/100
35/35 [==============================] - 52s 1s/step - loss: 0.7588 - val_loss: 0.6631
Epoch 6/100
35/35 [==============================] - 52s 1s/step - loss: 0.7420 - val_loss: 0.6498
Epoch 7/100
35/35 [==============================] - 52s 1s/step - loss: 0.7208 - val_loss: 0.6305
Epoch 8/100
35/35 [==============================] - 52s 1s/step - loss: 0.6963 - val_loss: 0.6297
Epoch 9/100
35/35 [==============================] - 51s 1s/step - loss: 0.6853 - val_loss: 0.6234
Epoch 10/100
35/35 [==============================] - 52s 1s/step - loss: 0.6641 - val_loss: 0.6085
Epoch 11/

**Bi-directional LSTM**

In [ ]:
input_layer = Input(shape = (max_len,))
embedding = Embedding(  
    len(word_index)+1, 300, weights = [embedding_matrix],
    input_length = max_len,trainable = False
    )(input_layer) 
x = SpatialDropout1D(0.3)(embedding)
x = Bidirectional(LSTM(300, dropout = 0.3, recurrent_dropout = 0.3))(x)

x = Dense(1024, activation = "relu")(x)
x = Dropout(0.8)(x)

x = Dense(1024, activation = "relu")(x)
x = Dropout(0.8)(x)

output = Dense(3, activation = "softmax")(x)

model = Model(inputs = input_layer, outputs = output)



Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 70)]              0         
_________________________________________________________________
embedding_7 (Embedding)      (None, 70, 300)           7783200   
_________________________________________________________________
spatial_dropout1d_6 (Spatial (None, 70, 300)           0         
_________________________________________________________________
bidirectional (Bidirectional (None, 600)               1442400   
_________________________________________________________________
dense_21 (Dense)             (None, 1024)              615424    
_________________________________________________________________
dropout_14 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_22 (Dense)             (None, 1024)              1049

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Fit the model with early stopping callback
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(xtrain_pad, y=ytrain_enc, batch_size=512, epochs=100, 
          verbose=1, validation_data=(xvalid_pad, yvalid_enc), callbacks=[earlystop])

Epoch 1/100
35/35 [==============================] - 295s 8s/step - loss: 1.0345 - val_loss: 0.8732
Epoch 2/100
35/35 [==============================] - 286s 8s/step - loss: 0.8623 - val_loss: 0.7427
Epoch 3/100
35/35 [==============================] - 280s 8s/step - loss: 0.7990 - val_loss: 0.7040
Epoch 4/100
35/35 [==============================] - 284s 8s/step - loss: 0.7741 - val_loss: 0.6966
Epoch 5/100
35/35 [==============================] - 288s 8s/step - loss: 0.7564 - val_loss: 0.6581
Epoch 6/100
35/35 [==============================] - 278s 8s/step - loss: 0.7318 - val_loss: 0.6454
Epoch 7/100
35/35 [==============================] - 276s 8s/step - loss: 0.7098 - val_loss: 0.6278
Epoch 8/100
35/35 [==============================] - 279s 8s/step - loss: 0.6814 - val_loss: 0.6324
Epoch 9/100
35/35 [==============================] - 278s 8s/step - loss: 0.6547 - val_loss: 0.6083
Epoch 10/100
35/35 [==============================] - 286s 8s/step - loss: 0.6268 - val_loss: 0.5977

# Ensemble

Abhishek은 그가 직접 만든 pysembler를 이용하고 있다.
logging 모듈을 공부해볼법한데, 일단은 패스!





In [1]:
#This is the main ensembling class.

import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold

import pandas as pd
import os 
import sys
import logging


)



SyntaxError: ignored